In [1]:
from dataloader import load_data_for_ann
from trainer import train_model, evaluate_model
from ann import ANNModelSimple
import torch
import pickle

In [ ]:
root_folder = "./tactile_dataset/"
file_name = "final_merged_df_sw500.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_losses_l = {}
train_accuracies_l = {}
val_losses_l = {}
val_accuracies_l = {}
test_accuracies_l = {}

for lr in [1e-4, 1e-3, 5e-3, 1e-2, 5e-2]:
    print(f"Training for lr={lr}")
    train_loader, val_loader, test_loader, num_outputs, num_features = load_data_for_ann(root_folder, file_name, batch_size=128, device=device)

    model = ANNModelSimple(num_features, num_outputs)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_losses, train_accuracies, val_losses, val_accuracies = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10, model_type='ann')
    train_losses_l[lr] = train_losses
    train_accuracies_l[lr] = train_accuracies
    val_losses_l[lr] = val_losses
    val_accuracies_l[lr] = val_accuracies

    test_accuracies_l[lr] = evaluate_model(model, test_loader)

out_file_name = 'exp_ann_lr_1e-4_5e-2_t_100_bs_128_ep_10.pkl'
with open(out_file_name, 'wb') as f:
    pickle.dump({"train_losses": train_losses_l, "train_accuracies": train_accuracies_l, "val_losses": val_losses_l, "val_accuracies": val_accuracies_l, "test_accuracies": test_accuracies_l}, f)
